In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
%matplotlib inline
# from functools import reduce

# set font
font_name = font_manager.FontProperties(
    fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font', family=font_name)
# matplotlib.rcParams['axes.unicode_minus'] = False
sns.set(font=font_name,
        rc={'axes.unicode_minus': False})  # to show negative sign on graphs

# set option
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.precision', 2)


In [50]:
# address standard
address = pd.read_csv('./data/processed/address.csv', encoding='cp949')

# customer dataset address
data = pd.read_csv('./data/raw/ALL_20220404_0cnt.csv')
temp = pd.read_csv('./data/raw/ALL_20220404_1cnt.csv')
data = pd.concat([data, temp])
temp = pd.read_csv('./data/raw/ALL_20220404_2cnt.csv')
data = pd.concat([data, temp])


In [84]:
check1 = set()
check2 = set()
# check by address
def check(dataframe):
    check_address = address['시도명']+' '+address['시군구명']
    check_dataframe = dataframe['시도명']+' '+dataframe['시군구명']
    result1 = set(check_address) - set(check_dataframe)
    check1.update(result1)
    result2 = set(check_dataframe) - set(check_address)
    check2.update(result2)
    return print(f'result1\n{result1}\n\nresult2\n{result2}')


In [51]:
# # feature selection
# columns = ['시도명', '시군구명', '읍면동']
# data = data[columns]

# sido = data['시도명'].unique()
# sigungu = data['시군구명'].unique()
# dong = data['읍면동'].unique()


In [52]:
# 시도명 dict
시도명사전 = {
    '인천': '인천광역시',
    '충남': '충청남도',
    '경기': '경기도',
    '대전': '대전광역시',
    '경남': '경상남도',
    '서울': '서울특별시',
    '전남': '전라남도',
    '전북': '전라북도',
    '제주': '제주특별자치도',
    '부산': '부산광역시',
    '광주': '광주광역시',
    '울산': '울산광역시',
    '경북': '경상북도',
    '충북': '충청북도',
    '대구': '대구광역시',
    '강원': '강원도',
}

# replace 시도명 based on 시도명 dict
data = data.replace({'시도명': 시도명사전})


In [53]:
# 세종특별자치시
세종_읍 = {'조치원읍'}
세종_면 = {'연기면', '연동면', '부강면', '금남면', '장군면', '연서면',
            '전의면', '전동면', '소정면'}
세종_행정동 = {'한솔동', '도담동', '해밀동', '아름동', '종촌동', '고운동',
            '보람동', '새롬동', '대평동', '소담동', '반곡동', '다정동'}
세종_법정동 = {'가람동', '한솔동', '도담동', '어진동', '해밀동',
            '산울동', '아름동', '종촌동', '고운동', '보람동', '나성동',
            '새롬동', '대평동', '소담동', '반곡동', '집현동', '합강동', '다정동'}
세종사전 = 세종_읍 | 세종_면 | 세종_행정동 | 세종_법정동

# replace 세종특별자치시 based on 세종_행정동 | 세종_법정동
data.loc[(data['읍면동'].isin(세종사전)) & (
    data['시도명'] == '충청남도'), '시도명'] = '세종특별자치시'
data.loc[(data['읍면동'].isin(세종사전)) & (
    data['시도명'] == '충청남도'), '시도명'] = '세종특별자치시'


In [81]:
data.loc[(data['시도명'] == '부산광역시') & (
    data['시군구명'].str.contains('진구')), '시군구명'] = '부산진구'
data.loc[data['시군구명'] == '강남', '시군구명'] = '강남구'
data.loc[data['시군구명'] == '강북', '시군구명'] = '강북구'

# 고양시 일산구
일산서구_행정동 = {'일산1동', '일산2동', '일산3동', '탄현1동', '탄현2동',
            '주엽1동', '주엽2동', '대화동', '송포동', '송산동'}
일산서구_법정동 = {'일산동', '탄현동', '주엽동', '대화동', '법곳동',
            '가좌동', '구산동', '덕이동'}
일산서구 = 일산서구_행정동 | 일산서구_법정동
일산동구_행정동 = {'식사동', '중산1동', '중산2동', '정발산동', '풍산동',
            '백석1동', '백석2동', '마두1동', '마두2동', '장항1동', '장항2동', '고봉동'}
일산동구_법정동 = {'식사동', '중산동', '정발산동', '풍동', '산황동', '백석동', '마두동',
            '장항동', '성석동', '설문동', '지영동', '문봉동', '사리현동'}
일산동구 = 일산동구_행정동 | 일산동구_법정동

data.loc[(data['시군구명'] == '고양시 일산구') & (
    data['읍면동'].isin(일산서구)), '시군구명'] = '고양시 일산서구'
data.loc[(data['시군구명'] == '고양시 일산구') & (
    data['읍면동'].isin(일산동구)), '시군구명'] = '고양시 일산동구'
# 일산4동 -> 정발산동
data.loc[data['시군구명'] == '고양시 일산구', '읍면동'] = '정발산동'
data.loc[data['시군구명'] == '고양시 일산구', '시군구명'] = '고양시 일산동구'


In [83]:
check(data)

result1
{'세종특별자치시 세종시'}

result2
{'경상남도 마산시 합포구', '세종특별자치시 연기군', '경기도 포천군', '경기도 ㆍ값없음', '충청남도 연기군', '인천광역시 미추홀구', '경기도 여주시', '경기도 광주군', '경상남도 마산시', '경상남도 창원시', '경기도 부천시', '충청북도 청주시 청원구', '충청남도 당진군', '경기도 과천시 원미구', '제주특별자치도 남제주군', '충청남도 천안시', '제주특별자치도 북제주군', '경기도 양주군', '경기도 용인시', '경기도 안산시', '충청북도 청주시 서원구', '경상남도 마산시 회원구', '경기도 화성군'}
